# 🎯 OTT Compliance Pipeline - Interactive Demo## Try the ML Compliance System with Sample DataThis notebook lets you interactively experience the OTT platform compliance monitoring system.###  1️⃣     2️⃣ GeoIP  - IP   3️⃣ ML   -   4️⃣   -  5️⃣   -   6️⃣   -  7️⃣ ROI  -   8️⃣   -  ---

## 1️⃣

In [ ]:
import sysfrom pathlib import Pathsys.path.insert(0, str(Path.cwd() / "src"))import pandas as pdimport jsonfrom datetime import datetime, timedeltafrom pprint import pprintfrom app.geoip_validator import geoip_validatorfrom app.ml_models import anomaly_detector, violation_predictorfrom app.adaptive_thresholds import adaptive_thresholdsfrom app.user_segments import user_segmentationfrom app.network_analysis import network_fraud_detectorfrom app.regulations import compliance_checker, RegulationFrameworkfrom app.roi_calculator import roi_calculatorfrom app.cache import cache_managerprint("✅  Module import !")print(f"Current time: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}")

## 2️⃣ GeoIP  - IP   IP     .

In [ ]:
print("="*70)print("🌍 GeoIP ")print("="*70)test_ips = [    {"ip": "8.8.8.8", "claimed_region": "US", "description": "Google DNS ()"},    {"ip": "1.1.1.1", "claimed_region": "AU", "description": "Cloudflare DNS ( )"},    {"ip": "185.220.101.1", "claimed_region": "US", "description": "Tor  ( )"},]results = []for test in test_ips:    print(f"\n📍 {test['description']}")    print(f"   IP: {test['ip']},  : {test['claimed_region']}")        result = geoip_validator.validate_ip_region_consistency(test["ip"], test["claimed_region"])        print(f"   : {result['flags'] if result['flags'] else ''}")    print(f"    : +{result['score_adjustment']}")    print(f"   VPN : {result['vpn_info']['is_vpn']}")        results.append({        "": test["description"],        "": "" if result['score_adjustment'] > 0 else "",        "": result['score_adjustment']    })print("\n" + "="*70)df = pd.DataFrame(results)print("📊 GeoIP   ")print(df.to_string(index=False))

## 3️⃣ ML   - Isolation Forest + LOF      .

In [ ]:
print("="*70)print("🤖 ML   ")print("="*70)sample_events = [    {        "name": "  ( )",        "features": {            "hour": 14,            "weekday": 2,            "event_type_len": 5,            "has_error": 0,            "is_eu": 0,            "has_consent": 1,            "subscription_tier": 2,            "device_id": 12345,            "region_code": 1,        }    },    {        "name": "  (  )",        "features": {            "hour": 3,            "weekday": 4,            "event_type_len": 8,            "has_error": 1,            "is_eu": 1,            "has_consent": 0,            "subscription_tier": 1,            "device_id": 99999,            "region_code": 50,        }    },    {        "name": "  (  +  )",        "features": {            "hour": 22,            "weekday": 6,            "event_type_len": 12,            "has_error": 1,            "is_eu": 1,            "has_consent": 0,            "subscription_tier": 1,            "device_id": 77777,            "region_code": 75,        }    },]ml_results = []for event in sample_events:    print(f"\n📌 {event['name']}")    result = anomaly_detector.ensemble_anomaly_detection(event['features'])        print(f"    : {'⚠️  YES' if result['is_anomaly'] else '✅ NO'}")    print(f"    : {result['ensemble_score']:.3f}")    print(f"   Isolation Forest: {result['isolation_forest']['score']:.3f}")    print(f"   LOF Score: {result['lof']['score']:.3f}")    print(f"   : {result['flags'] if result['flags'] else ''}")        ml_results.append({        "": event['name'],        " ": "⚠️  YES" if result['is_anomaly'] else "✅ NO",        " ": f"{result['ensemble_score']:.3f}",    })print("\n" + "="*70)df_ml = pd.DataFrame(ml_results)print("📊 ML   ")print(df_ml.to_string(index=False))print(f"\n📈   : {len(anomaly_detector.feature_history)}")print("   (100     )")

## 4️⃣   -       6   .

In [ ]:
print("="*70)print("👥  ")print("="*70)user_profiles = [    {        "user_id": "power_user_001",        "event_count_30d": 850,        "event_count_7d": 160,        "violation_count_30d": 0,        "days_since_signup": 250,        "last_activity_days_ago": 1,        "avg_risk_score": 2.0,    },    {        "user_id": "new_user_002",        "event_count_30d": 15,        "event_count_7d": 8,        "violation_count_30d": 0,        "days_since_signup": 10,        "last_activity_days_ago": 2,        "avg_risk_score": 3.5,    },    {        "user_id": "suspicious_user_003",        "event_count_30d": 120,        "event_count_7d": 90,        "violation_count_30d": 10,        "days_since_signup": 45,        "last_activity_days_ago": 1,        "avg_risk_score": 9.0,    },    {        "user_id": "inactive_user_004",        "event_count_30d": 35,        "event_count_7d": 2,        "violation_count_30d": 0,        "days_since_signup": 120,        "last_activity_days_ago": 60,        "avg_risk_score": 4.0,    },]segment_results = []for profile in user_profiles:    user_id = profile.pop("user_id")    segment = user_segmentation.update_user_profile(user_id=user_id, **profile)    params = user_segmentation.get_segment_risk_parameters(segment)        print(f"\n👤 {user_id}")    print(f"   : {segment.value}")    print(f"   : {params['risk_threshold_high']}")    print(f"   : {params['anomaly_sensitivity']}x")    print(f"   : {', '.join(params['alert_channels'])}")        segment_results.append({        "": user_id,        "": segment.value,        "": params['risk_threshold_high'],        "": f"{params['anomaly_sensitivity']}x",    })print("\n" + "="*70)df_segments = pd.DataFrame(segment_results)print("📊   ")print(df_segments.to_string(index=False))

## 5️⃣   -    /IP/      .

In [ ]:
print("="*70)print("🔗   ")print("="*70)fraud_network = [    ("fraud_user_1", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("fraud_user_2", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("fraud_user_3", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("fraud_user_4", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("fraud_user_5", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("fraud_user_6", "device_A", "192.168.1.100", "visa_xxxx_1234"),    ("clean_user_1", "device_B", "192.168.1.200", "visa_xxxx_5678"),    ("clean_user_2", "device_C", "192.168.1.300", "mastercard_1111"),]print("\n📌  8   ...")for user_id, device_id, ip_address, payment_method in fraud_network:    network_fraud_detector.add_user_event(        user_id=user_id,        device_id=device_id,        ip_address=ip_address,        payment_method=payment_method    )print("✅ Complete")print("\n🔍    ...")rings = network_fraud_detector.detect_fraud_rings(min_ring_size=5)print(f"\n⚠️  {len(rings)}   !\n")for i, ring in enumerate(rings, 1):    print(f"   🔴      print(f"      : {len(ring['users'])}")    print(f"      : {ring['risk_score']:.2f}")    print(f"       : {', '.join(ring['users'][:3])}...")    print()stats = network_fraud_detector.get_network_statistics()print("📊  ")print(f"    : {stats['total_nodes']}")print(f"    : {stats['total_edges']}")print(f"     : {stats['detected_fraud_rings']}")print(f"      : {stats['users_in_fraud_rings']}")print("\n👤   ")test_users = ["fraud_user_1", "fraud_user_2", "clean_user_1"]for user in test_users:    risk = network_fraud_detector.get_user_network_risk(user)    print(f"   {user}: {risk['risk_score']:.2f} {'🔴 HIGH' if risk['risk_score'] > 0.5 else '🟢 LOW'}")

## 6️⃣    -  GDPR, CCPA, PIPL  10      .

In [ ]:
print("="*70)print("⚖️     ")print("="*70)print("\n🌍   :")regions = ["EU", "US", "CN", "BR", "TH", "AU"]for region in regions:    regs = RegulationFramework.get_regulations_for_region(region)    reg_names = [r.value for r in regs] if regs else ""    print(f"   {region}: {reg_names}")print("\n📋   :")from app.regulations import Regulationfor reg in [Regulation.GDPR, Regulation.CCPA, Regulation.PIPL]:    reqs = RegulationFramework.get_regulation_requirements(reg)    print(f"\n   {reg.value}:")    print(f"       : {'' if reqs['consent_required'] else ''}")    print(f"       : {reqs['breach_notification_days']}")    print(f"       : {reqs['max_retention_years']}")    print(f"       : {'' if reqs['right_to_deletion'] else ''}")    print(f"       : {'' if reqs['data_portability'] else ''}")print("\n\n✅   :")test_events = [    {        "user_id": "user_eu_001",        "event_type": "user_data_access",        "region": "EU",        "details": {"access_time_days": 2, "has_explicit_consent": True},        "description": "EU    (2 , )"    },    {        "user_id": "user_us_001",        "event_type": "user_data_deletion",        "region": "US",        "details": {"deletion_time_days": 5, "has_explicit_consent": True},        "description": "    (5 )"    },    {        "user_id": "user_cn_001",        "event_type": "user_consent_change",        "region": "CN",        "details": {"has_explicit_consent": False},        "description": "    ()"    },]for event in test_events:    print(f"\n   {event['description']}")    result = compliance_checker.evaluate_event_compliance(        user_id=event['user_id'],        event_type=event['event_type'],        region=event['region'],        event_details=event['details']    )        status = "✅ " if result['compliant'] else "❌ "    print(f"      : {status}")    print(f"       : {', '.join(result['applicable_regulations'])}")    if result['violations']:        for v in result['violations']:            print(f"      ⚠️   : {v['reason']}")

## 7️⃣ ROI  -       .

In [ ]:
print("="*70)print("💰 ROI  -    ")print("="*70)print("\n📊 : 12  100,000  ")print("   -  : 100")print("   -  : 80")print("   -  : 3")report = roi_calculator.generate_roi_report(    violations_detected=100,    violations_prevented=80,    incidents_prevented=3,    total_users=100000,    customer_lifetime_value=500,    time_period_months=12,    applicable_regulations=["GDPR", "CCPA", "PIPL", "LGPD"])print("\n💵   :")summary = report['financial_summary']print(f"\n     : ${summary['total_value_protected']:,}")print(f"    : ${summary['system_cost']:,}")print(f"    : ${summary['net_benefit']:,}")print(f"   ROI: {summary['roi_percent']:,.0f}%")print(f"    : {summary['payback_period_months']:.1f}")print("\n⚖️    :")for reg, fine_data in report['fine_prevention'].items():    print(f"   {reg}: ${fine_data['total_value']:,}")print("\n👥   :")rep = report['reputation_protection']print(f"     : {rep['customers_protected']:,}")print(f"    : ${rep['value_per_incident']:,}")print(f"    : ${rep['total_protection_value']:,}")print("\n📈   :")monthly = report['monthly_breakdown']print(f"    :")for cost_type, amount in monthly['expenses'].items():    print(f"      {cost_type}: ${amount:,.0f}")print(f"\n    :")for saving_type, amount in monthly['savings'].items():    print(f"      {saving_type}: ${amount:,.0f}")

## 8️⃣   -  , ,       .

In [ ]:
print("="*70)print("📊   -    ")print("="*70)test_cases = [    {"hour": 2, "region": "EU", "segment": "new_user", "desc": "(2), EU,  "},    {"hour": 9, "region": "US", "segment": "normal_user", "desc": "(9), US,  "},    {"hour": 14, "region": "CN", "segment": "power_user", "desc": "(14), CN,  "},    {"hour": 22, "region": "BR", "segment": "suspicious_user", "desc": "(22), BR,  "},]threshold_results = []print("\n📌    :\n")for case in test_cases:    desc = case.pop("desc")    threshold = adaptive_thresholds.get_dynamic_risk_threshold(**case)        print(f"   {desc}")    print(f"      →  : {threshold:.2f}")        threshold_results.append({        "//": desc,        "": f"{threshold:.2f}",    })print("\n\n📚    :")learning_events = [    {"risk_score": 3.0, "is_violation": False, "segment": "normal_user", "hour": 10, "region": "US"},    {"risk_score": 7.5, "is_violation": True, "segment": "new_user", "hour": 2, "region": "EU"},    {"risk_score": 9.0, "is_violation": True, "segment": "suspicious_user", "hour": 22, "region": "BR"},]for i, event in enumerate(learning_events, 1):    adaptive_thresholds.record_event(**event)    print(f"    {i}: Risk={event['risk_score']:.1f}, "          f"Violation={'Yes' if event['is_violation'] else 'No'}, "          f"Segment={event['segment']}")print("\n✅     .")print("      .")

## 9️⃣   -           .

In [ ]:
print("="*70)print("🔄   -    ")print("="*70)sample_event = {    "event_id": "evt_20260113_001",    "user_id": "user_eu_fraud_001",    "device_id": "device_A",    "ip_address": "185.220.101.45",    "timestamp": datetime.utcnow().isoformat(),    "region": "EU",    "is_eu": True,    "has_consent": False,    "event_type": "bulk_export",    "error_code": "AUTH_FAILED",    "subscription_plan": "basic",    "extra_metadata": {"export_size": "50GB", "records": 10000}}print("\n📥  :")print(f"   Event ID: {sample_event['event_id']}")print(f"   User: {sample_event['user_id']}")print(f"   Event Type: {sample_event['event_type']}")print(f"   Region: {sample_event['region']}")print("\n" + "="*70)print("🔍   :")print("="*70)print("\n1️⃣  GeoIP ")geoip_result = geoip_validator.validate_ip_region_consistency(    sample_event['ip_address'],    sample_event['region'])print(f"   IP  : {'✅ YES' if not geoip_result['flags'] else '❌ NO'}")print(f"    : +{geoip_result['score_adjustment']}")print("\n2️⃣  ML  ")ml_features = {    "hour": datetime.utcnow().hour,    "weekday": datetime.utcnow().weekday(),    "event_type_len": len(sample_event['event_type']),    "has_error": 1 if sample_event['error_code'] else 0,    "is_eu": int(sample_event['is_eu']),    "has_consent": int(sample_event['has_consent']),    "subscription_tier": 1,    "device_id": hash(sample_event['device_id']) % 1000,    "region_code": hash(sample_event['region']) % 100,}ml_result = anomaly_detector.ensemble_anomaly_detection(ml_features)print(f"    : {'⚠️  YES' if ml_result['is_anomaly'] else '✅ NO'}")print(f"    : {ml_result['ensemble_score']:.3f}")print("\n3️⃣   ")segment = user_segmentation.get_user_segment(sample_event['user_id'])params = user_segmentation.get_segment_risk_parameters(segment)print(f"   : {segment.value}")print(f"   : {params['risk_threshold_high']}")print(f"   : {params['anomaly_sensitivity']}x")print("\n4️⃣   ")network_fraud_detector.add_user_event(    user_id=sample_event['user_id'],    device_id=sample_event['device_id'],    ip_address=sample_event['ip_address'])network_risk = network_fraud_detector.get_user_network_risk(sample_event['user_id'])print(f"    : {network_risk['risk_score']:.2f}")if network_risk['risk_factors']:    print(f"    : {', '.join(network_risk['risk_factors'])}")print("\n5️⃣    ")compliance_result = compliance_checker.evaluate_event_compliance(    user_id=sample_event['user_id'],    event_type=sample_event['event_type'],    region=sample_event['region'],    event_details={"has_explicit_consent": sample_event['has_consent']})print(f"    : {'✅ YES' if compliance_result['compliant'] else '❌ NO'}")if not compliance_result['compliant']:    for v in compliance_result['violations']:        print(f"   ⚠️  {v['regulation']}: {v['reason']}")print("\n" + "="*70)print("📊   ")print("="*70)final_risk_score = (    5 +    geoip_result['score_adjustment'] +    (ml_result['ensemble_score'] * 2) +    (network_risk['risk_score'] * 2) +    (2 if not compliance_result['compliant'] else 0))risk_level = "🔴 HIGH" if final_risk_score >= 8 else ("🟡 MEDIUM" if final_risk_score >= 5 else "🟢 LOW")print(f"\n  : {final_risk_score:.2f}")print(f" : {risk_level}")print(f"\n : {'⏸️   ' if final_risk_score >= 8 else '📋 ' if final_risk_score >= 5 else '✅ '}")

## 🎯 ### 10   ```📥     ↓🌍 GeoIP  → IP/    ↓🤖 ML   → Isolation Forest + LOF   ↓👥   →      ↓🔗   →      ↓⚖️   → GDPR/CCPA/PIPL    ↓📊   →      ↓💰 ROI  →     ↓🚨   →     ↓💾  →     ↓⏰   →  ```###  - **  **:     - ** **: 6    - ** **:      - ** **: 10 /   - **ROI**:  $1.3B+  - ****: Redis  <100ms  - ****:      ---**🎉 ! OTT Compliance Pipeline   .** :   FastAPI  !```bashpython -m uvicorn src.app.main:app --reload```